# Advanced and Specialized Visualizations

This notebook covers PanelBox's specialized visualization modules:

- **Part 1:** VAR Visualizations — IRFs, FEVD, Stability Diagram, Granger Causality Network
- **Part 2:** Quantile Regression Visualizations — Coefficient Paths β(τ), Fan Charts, Interactive Dashboard
- **Part 3:** Spatial Visualizations — Moran Scatterplot, Spatial Weights Network, Choropleth
- **Part 4:** Model Comparison — Coefficient Plots, Forest Plots, IC Comparison

**Prerequisites:** Notebooks 01–02 of this series; familiarity with VAR, Quantile, and Spatial models  
**Duration:** ~150–180 minutes

In [ ]:
import sys
import os
sys.path.insert(0, '../../../')
sys.path.insert(0, '../')

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default = 'notebook'

np.random.seed(42)

# Local generators
from utils.data_generators import (
    make_var_data,
    make_quantile_wage_panel,
    make_spatial_data,
    generate_panel_data,
)

# Output directories
os.makedirs('../outputs/charts/png', exist_ok=True)
os.makedirs('../outputs/charts/svg', exist_ok=True)
os.makedirs('../outputs/reports/html', exist_ok=True)

print('Setup complete.')

## Part 1: VAR (Vector Autoregression) Visualizations

VAR models capture dynamic interactions between multiple time series.
PanelBox provides specialized plots for:
- **Impulse Response Functions (IRFs)** — how a shock to one variable propagates
- **Forecast Error Variance Decomposition (FEVD)** — which shocks drive each variable's variance
- **Stability Diagram** — are eigenvalues inside the unit circle?
- **Granger Causality Network** — directed graph of significant causal relationships

We use a simulated system: GDP Growth, Inflation, Interest Rate (quarterly, 200 obs).

In [ ]:
from panelbox.var import PanelVAR
from panelbox.var.data import PanelVARData

# Generate VAR data — time series (quarterly macroeconomic variables)
df_var = make_var_data(n_obs=200, n_vars=3, seed=42)
print(df_var.head())
print(f'\nShape: {df_var.shape}')
print(f'Variables: {df_var.columns.tolist()}')
print(f'Date range: {df_var.index[0]} to {df_var.index[-1]}')

In [ ]:
# PanelVARData requires entity + time columns.
# make_var_data returns a DatetimeIndex time series — treat it as a single-entity panel.
df_var_panel = df_var.copy().reset_index()
df_var_panel.columns = ['period'] + list(df_var.columns)
df_var_panel['entity'] = 1  # single entity (pure time series)

var_data = PanelVARData(
    data=df_var_panel,
    endog_vars=['gdp_growth', 'inflation', 'interest_rate'],
    entity_col='entity',
    time_col='period',
    lags=2,
    trend='constant',
)

var_model = PanelVAR(data=var_data)
var_results = var_model.fit(cov_type='nonrobust')  # single entity: cluster not meaningful
print('VAR(2) estimated.')
print(var_results.summary())

### 1.1 Impulse Response Functions (IRFs)

An IRF shows how a one-standard-deviation shock to variable *i* propagates through
the system over *h* periods. The full 3×3 grid shows all impulse–response combinations.

In [ ]:
from panelbox.visualization.var_plots import plot_irf

# Compute IRFs — Cholesky decomposition, no bootstrap (faster)
irf_result = var_results.irf(
    periods=20,
    method='cholesky',
    ci_method=None,   # set to 'bootstrap' for confidence bands (slower)
)

# Plot the full IRF grid (3×3 for a 3-variable VAR)
fig_irf = plot_irf(
    irf_result=irf_result,
    ci=False,
    backend='matplotlib',
    theme='academic',
    show=False,
)

if fig_irf is not None:
    fig_irf.tight_layout()
    try:
        fig_irf.savefig('../outputs/charts/png/03_var_irf.png', dpi=150, bbox_inches='tight')
        print('IRF chart saved: ../outputs/charts/png/03_var_irf.png')
    except Exception as e:
        print(f'Save failed: {e}')
    plt.show()
else:
    print('IRF figure not returned (show=True consumed it). Re-run with show=False.')

In [ ]:
# --- Exercise 1 ---
# Re-compute IRFs with bootstrap confidence bands (n_bootstrap=200 for speed)
# and plot only the response of 'gdp_growth' to shocks (use response= parameter).

# YOUR CODE HERE


### 1.2 Forecast Error Variance Decomposition (FEVD)

FEVD decomposes the forecast error variance of each variable into contributions from
each structural shock. Stacked area charts show the proportion explained at each horizon.

In [ ]:
from panelbox.visualization.var_plots import plot_fevd

# Compute FEVD
fevd_result = var_results.fevd(periods=20, method='cholesky')

# Plot FEVD as stacked area charts (one panel per variable)
fig_fevd = plot_fevd(
    fevd_result=fevd_result,
    kind='area',
    backend='matplotlib',
    theme='academic',
    show=False,
)

if fig_fevd is not None:
    fig_fevd.tight_layout()
    try:
        fig_fevd.savefig('../outputs/charts/png/03_var_fevd.png', dpi=150, bbox_inches='tight')
        print('FEVD chart saved: ../outputs/charts/png/03_var_fevd.png')
    except Exception as e:
        print(f'Save failed: {e}')
    plt.show()
else:
    print('FEVD figure not returned.')

### 1.3 Stability Diagram — Unit Circle

A VAR(p) is stable if all eigenvalues of the companion matrix lie **inside** the unit circle
(modulus < 1). The stability diagram plots all eigenvalues against the unit circle.

In [ ]:
# Eigenvalues of the companion matrix
eigenvalues = var_results.eigenvalues
moduli = np.abs(eigenvalues)
print(f'Eigenvalue moduli: {moduli.round(4)}')
print(f'Maximum modulus: {moduli.max():.4f}')
print(f'System stable (max|λ| < 1): {moduli.max() < 1}')

# Stability plot — uses the eigenvalues property directly
fig_stab = var_results.plot_stability(backend='matplotlib', show=False)

if fig_stab is not None:
    fig_stab.tight_layout()
    try:
        fig_stab.savefig('../outputs/charts/png/03_var_stability.png', dpi=150, bbox_inches='tight')
        print('Stability chart saved: ../outputs/charts/png/03_var_stability.png')
    except Exception as e:
        print(f'Save failed: {e}')
    plt.show()
else:
    # Fallback: draw manually
    fig, ax = plt.subplots(figsize=(6, 6))
    theta = np.linspace(0, 2 * np.pi, 300)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', lw=1.5, label='Unit circle')
    ax.axhline(0, color='gray', lw=0.5)
    ax.axvline(0, color='gray', lw=0.5)
    colors = ['green' if m < 1 else 'red' for m in moduli]
    ax.scatter(eigenvalues.real, eigenvalues.imag, c=colors, s=80, zorder=5)
    ax.set_xlim(-1.3, 1.3); ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.set_title('VAR Stability — Roots of Companion Matrix')
    ax.set_xlabel('Real part'); ax.set_ylabel('Imaginary part')
    plt.tight_layout()
    fig.savefig('../outputs/charts/png/03_var_stability.png', dpi=150, bbox_inches='tight')
    plt.show()

### 1.4 Granger Causality Network

Granger causality tests whether lags of variable *X* help predict variable *Y* beyond
what *Y*'s own lags predict. The network diagram shows significant causal links.

In [ ]:
# Compute all pairwise Granger causality p-values
gc_matrix = var_results.granger_causality_matrix(significance_level=0.05)
print('Granger Causality p-value matrix:')
print('(Row = cause, Column = effect; p < 0.05 → significant)')
print(gc_matrix.round(4))

In [ ]:
# Plot Granger causality network — directed graph (Plotly, interactive)
# Edges drawn where p-value < threshold
fig_network = var_results.plot_causality_network(
    threshold=0.10,    # show edges with p < 0.10 (relaxed for illustration)
    layout='circular',
    backend='plotly',
    show=True,
)

In [ ]:
# Test individual pairs explicitly
variables = ['gdp_growth', 'inflation', 'interest_rate']
print('Pairwise Granger causality tests (VAR(2)):')
print('=' * 60)
for cause in variables:
    for effect in variables:
        if cause != effect:
            gc = var_results.granger_causality(cause=cause, effect=effect)
            sig = '***' if gc.p_value < 0.01 else ('**' if gc.p_value < 0.05 else
                  ('*' if gc.p_value < 0.10 else ''))
            print(f'  {cause} → {effect}: F={gc.f_stat:.3f}, p={gc.p_value:.4f} {sig}')

## Part 2: Quantile Regression Visualizations

Quantile regression estimates conditional quantiles τ ∈ (0, 1), revealing how effects
vary across the outcome distribution.

Key visualizations:
- **Coefficient Path β(τ)** — how a coefficient changes from τ = 0.10 to τ = 0.90
- **Fan Chart** — predicted quantile bands across a covariate range
- **Interactive Dashboard** — Plotly-powered exploration by quantile

We use a simulated wage panel where the education effect is **heterogeneous across quantiles**.

In [ ]:
from panelbox.models.quantile import PooledQuantile

df_wage = make_quantile_wage_panel(n_entities=200, n_periods=5, seed=42)
df_wage_reset = df_wage.reset_index()
print(df_wage_reset.head())
print(f'\nShape: {df_wage_reset.shape}')
print(f'Columns: {df_wage_reset.columns.tolist()}')
print(f'Wage range: [{df_wage_reset["wage"].min():.2f}, {df_wage_reset["wage"].max():.2f}]')
print(f'Education range: [{df_wage_reset["educ"].min()}, {df_wage_reset["educ"].max()}] years')

In [ ]:
# Fit PooledQuantile at 5 quantile levels simultaneously
# exog must include an intercept column (PooledQuantile does not add it automatically)
quantiles = np.array([0.10, 0.25, 0.50, 0.75, 0.90])

X_wages = df_wage_reset[['educ', 'exper']].values
y_wages = df_wage_reset['wage'].values
X_with_const = np.column_stack([np.ones(len(y_wages)), X_wages])

qr_model = PooledQuantile(
    endog=y_wages,
    exog=X_with_const,
    entity_id=df_wage_reset['entity'].values,
    time_id=df_wage_reset['time'].values,
    quantiles=quantiles,
)
qr_result = qr_model.fit(se_type='cluster')

# params shape: (n_params, n_quantiles) = (3, 5)
param_names = ['Intercept', 'Education', 'Experience']
print(f'params shape: {qr_result.params.shape}  (n_params × n_quantiles)')
print()
params_df = pd.DataFrame(
    qr_result.params,
    index=param_names,
    columns=[f'τ={q:.2f}' for q in quantiles]
)
print('Estimated coefficients by quantile:')
print(params_df.round(4))

### 2.1 Coefficient Path β(τ)

The coefficient path shows how each regressor's effect changes across the quantile
distribution. An upward slope for *Education* means high-wage workers benefit more
from each additional year of schooling.

In [ ]:
from panelbox.visualization.quantile.advanced_plots import QuantileVisualizer

try:
    qv = QuantileVisualizer(style='academic', dpi=150)

    # Coefficient path for all variables (one subplot per variable)
    fig_coef_path = qv.coefficient_path(
        result=qr_result,
        var_names=param_names,
        uniform_bands=True,
    )
    fig_coef_path.suptitle('Coefficient Paths Across Quantiles β(τ)', y=1.02)
    fig_coef_path.tight_layout()
    try:
        fig_coef_path.savefig('../outputs/charts/png/03_qr_coef_path.png',
                              dpi=150, bbox_inches='tight')
        print('Coefficient path saved: ../outputs/charts/png/03_qr_coef_path.png')
    except Exception as e:
        print(f'Save failed: {e}')
    plt.show()
except Exception as e:
    print(f'QuantileVisualizer.coefficient_path error: {e}')
    print('Falling back to manual plot.')
    # Manual coefficient path
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    for idx, (ax, name) in enumerate(zip(axes, param_names)):
        coefs = qr_result.params[idx, :]  # shape (n_quantiles,)
        ax.plot(quantiles, coefs, 'o-', color='steelblue', lw=2, ms=6)
        ax.axhline(np.mean(coefs), color='red', ls='--', lw=1, label='Mean')
        ax.set_title(name); ax.set_xlabel('Quantile τ'); ax.set_ylabel('Coefficient')
        ax.legend(fontsize=9)
    fig.suptitle('Coefficient Paths Across Quantiles β(τ)')
    fig.tight_layout()
    fig.savefig('../outputs/charts/png/03_qr_coef_path.png', dpi=150, bbox_inches='tight')
    plt.show()

**Interpretation of the Education coefficient path:**

- Workers at the **top** of the wage distribution (τ = 0.90) benefit **more** from each
  additional year of education than workers at the bottom (τ = 0.10)
- OLS estimates only the **average** effect and misses this distributional heterogeneity
- An upward-sloping β(τ) for Education confirms that returns to education are **unequal**

### 2.2 Fan Chart — Distributional Forecast

A fan chart shows the predicted wage at each quantile as a function of education level,
holding experience fixed at its mean. The spread of the fan reveals conditional dispersion.

In [ ]:
educ_range = np.linspace(8, 20, 60)
avg_exper  = df_wage_reset['exper'].mean()

# X_forecast: shape (n_points, n_params) — one row per education value
X_forecast = np.column_stack([
    np.ones(len(educ_range)),
    educ_range,
    np.full(len(educ_range), avg_exper),
])

try:
    if 'qv' not in dir():
        from panelbox.visualization.quantile.advanced_plots import QuantileVisualizer
        qv = QuantileVisualizer(style='academic', dpi=150)
    fig_fan = qv.fan_chart(
        result=qr_result,
        X_forecast=X_forecast,
        time_index=educ_range,
        tau_list=list(quantiles),
        alpha_gradient=True,
    )
    fig_fan.axes[0].set_xlabel('Years of Education')
    fig_fan.axes[0].set_ylabel('Predicted Wage')
    fig_fan.axes[0].set_title('Wage Distribution by Education Level — Fan Chart')
    fig_fan.tight_layout()
    try:
        fig_fan.savefig('../outputs/charts/png/03_qr_fan_chart.png', dpi=150, bbox_inches='tight')
        print('Fan chart saved: ../outputs/charts/png/03_qr_fan_chart.png')
    except Exception as e:
        print(f'Save failed: {e}')
    plt.show()
except Exception as e:
    print(f'fan_chart error: {e}  — using manual Plotly fan chart.')
    import plotly.graph_objects as go
    # Manually compute predictions: y_hat = X_forecast @ params[:, q_idx]
    colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4', '#9467bd']
    fig_fan_plotly = go.Figure()
    for i, tau in enumerate(quantiles):
        y_pred = X_forecast @ qr_result.params[:, i]
        fig_fan_plotly.add_trace(go.Scatter(
            x=educ_range, y=y_pred,
            mode='lines', name=f'τ = {tau:.2f}',
            line=dict(color=colors[i], width=2),
        ))
    fig_fan_plotly.update_layout(
        title='Wage Distribution by Education Level — Fan Chart',
        xaxis_title='Years of Education',
        yaxis_title='Predicted Wage',
        template='plotly_white',
        font=dict(family='Arial', size=13),
    )
    fig_fan_plotly.show()

### 2.3 Interactive Coefficient Dashboard

The `InteractivePlotter` builds a Plotly dashboard where you can explore all coefficient
paths simultaneously with hover tooltips and zoom.

In [ ]:
from panelbox.visualization.quantile.interactive import InteractivePlotter

try:
    ip = InteractivePlotter(theme='plotly_white')
    dash_fig = ip.coefficient_dashboard(
        result=qr_result,
        var_names=param_names,
    )
    dash_fig.show()
    dash_fig.write_html(
        '../outputs/reports/html/03_quantile_dashboard.html',
        include_plotlyjs=True,
    )
    print('Interactive dashboard saved: ../outputs/reports/html/03_quantile_dashboard.html')
except Exception as e:
    print(f'InteractivePlotter error: {e}  — using manual Plotly slider.')
    import plotly.graph_objects as go
    # Manual slider: one bar trace per quantile, toggled by animation slider
    frames = []
    for i, tau in enumerate(quantiles):
        frames.append(go.Frame(
            data=[go.Bar(x=param_names, y=qr_result.params[:, i])],
            name=f'{tau:.2f}',
        ))

    fig_slider = go.Figure(
        data=[go.Bar(x=param_names, y=qr_result.params[:, 2],
                     marker_color='steelblue')],  # start at τ=0.50
        frames=frames,
    )
    fig_slider.update_layout(
        title='Coefficients by Quantile',
        xaxis_title='Variable',
        yaxis_title='Coefficient',
        template='plotly_white',
        font=dict(family='Arial', size=13),
        sliders=[{
            'steps': [
                {'method': 'animate', 'label': f'τ={tau:.2f}',
                 'args': [[f'{tau:.2f}'], {'frame': {'duration': 0}, 'mode': 'immediate'}]}
                for tau in quantiles
            ],
            'currentvalue': {'prefix': 'Quantile τ = '},
            'pad': {'t': 50},
        }],
    )
    fig_slider.show()
    fig_slider.write_html(
        '../outputs/reports/html/03_quantile_dashboard.html',
        include_plotlyjs=True,
    )
    print('Interactive dashboard saved: ../outputs/reports/html/03_quantile_dashboard.html')

In [ ]:
# --- Exercise 2 ---
# Use SurfacePlotter to create an interactive surface or heatmap of coefficients
# across quantiles and variables.
# Hint: from panelbox.visualization.quantile.surface_plots import SurfacePlotter
#       sp = SurfacePlotter(); sp.coefficient_heatmap(result=qr_result, var_names=param_names)

# YOUR CODE HERE


## Part 3: Spatial Visualizations

Spatial econometrics requires specialized visualizations to understand geographic patterns.

Key charts:
- **Moran Scatterplot** — standardized values vs. spatial lag; slope ≈ Moran's I
- **Spatial Weights Network** — visualize the W matrix as a geographic graph
- **Choropleth Map** — geographic distribution of values (requires `geopandas`)

We use simulated house price data with x–y coordinates.

> **Note:** The choropleth map section requires `geopandas`. Install with:
> `pip install geopandas`

In [ ]:
from scipy.spatial.distance import cdist

# Generate spatial data — house prices with coordinates
df_spatial = make_spatial_data(n_obs=100, seed=42)
print(df_spatial.head())
print(f'\nShape: {df_spatial.shape}')
print(f'Price range: [{df_spatial["price"].min():.1f}, {df_spatial["price"].max():.1f}]')

# Build distance-based spatial weights matrix (threshold = 2 units)
coords    = df_spatial[['x', 'y']].values
dist_mat  = cdist(coords, coords)
W_raw     = (dist_mat < 2.0).astype(float)
np.fill_diagonal(W_raw, 0)

# Row-standardize so each row sums to 1
row_sums  = W_raw.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1   # avoid division by zero for isolated observations
W         = W_raw / row_sums

print(f'\nW matrix shape: {W.shape}')
print(f'Average number of neighbors: {(W > 0).sum(axis=1).mean():.1f}')

### 3.1 Moran Scatterplot

The Moran scatterplot plots standardized values (z) on the x-axis against their spatial
lag (Wz) on the y-axis. The slope of the regression line equals **Moran's I**.

The four quadrants identify spatial clusters:
- **HH** (High-High): high-value surrounded by high-value neighbors
- **LL** (Low-Low): low-value surrounded by low-value neighbors
- **HL / LH**: spatial outliers

In [ ]:
from panelbox.visualization.spatial_plots import create_moran_scatterplot

fig_moran = create_moran_scatterplot(
    values=df_spatial['price'].values,
    W=W,
    title='Moran Scatterplot — House Prices',
    xlabel='Standardized Price',
    ylabel='Spatial Lag of Standardized Price',
    show_regression=True,
    show_quadrants=True,
)
fig_moran.tight_layout()
try:
    fig_moran.savefig('../outputs/charts/png/03_spatial_moran.png', dpi=150, bbox_inches='tight')
    print('Moran scatterplot saved: ../outputs/charts/png/03_spatial_moran.png')
except Exception as e:
    print(f'Save failed: {e}')
plt.show()

In [ ]:
# Verify Moran's I manually
prices   = df_spatial['price'].values
z        = (prices - prices.mean()) / prices.std()
Wz       = W @ z
n        = len(z)
S0       = W.sum()         # sum of all weights
morans_i = (n / S0) * (z @ Wz) / (z @ z)
print(f"Moran's I = {morans_i:.4f}")
print("Positive value → positive spatial autocorrelation (similar prices cluster together)")

### 3.2 Spatial Weights Network

The network visualizes which observations are connected (neighbors) according to the
spatial weights matrix W. Node colors represent the house price level.

In [ ]:
import plotly.graph_objects as go

# Build edge traces from the W matrix
edge_x, edge_y = [], []
for i in range(len(coords)):
    for j in range(i + 1, len(coords)):   # upper triangle only (undirected)
        if W[i, j] > 0:
            edge_x.extend([coords[i, 0], coords[j, 0], None])
            edge_y.extend([coords[i, 1], coords[j, 1], None])

fig_net = go.Figure()

fig_net.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    mode='lines',
    line=dict(width=0.6, color='#aaa'),
    hoverinfo='none',
    name='Neighbors',
    showlegend=False,
))

fig_net.add_trace(go.Scatter(
    x=coords[:, 0], y=coords[:, 1],
    mode='markers',
    marker=dict(
        size=9,
        color=df_spatial['price'].values,
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title='Price (€k)'),
        line=dict(width=0.5, color='white'),
    ),
    hovertemplate='Price: %{marker.color:.1f}<br>x: %{x:.2f}, y: %{y:.2f}<extra></extra>',
    name='Houses',
))

fig_net.update_layout(
    title='Spatial Weights Network — House Prices',
    xaxis_title='X coordinate', yaxis_title='Y coordinate',
    template='plotly_white',
    font=dict(family='Arial', size=12),
    width=750, height=600,
)
fig_net.show()

### 3.3 Spatial Weights Structure — Heatmap

In [ ]:
from panelbox.visualization.spatial_plots import plot_spatial_weights_structure

# Show first 30×30 submatrix for readability
fig_wstruct = plot_spatial_weights_structure(
    W=W[:30, :30],
    title='Spatial Weights Matrix W (first 30 observations)',
    figsize=(8, 8),
    cmap='Blues',
    show_colorbar=True,
)
fig_wstruct.tight_layout()
plt.show()

### 3.4 Choropleth Map (optional — requires `geopandas`)

In [ ]:
try:
    import geopandas as gpd
    from shapely.geometry import Point

    gdf = gpd.GeoDataFrame(
        df_spatial,
        geometry=[Point(x, y) for x, y in zip(df_spatial['x'], df_spatial['y'])],
        crs='EPSG:4326',
    )
    fig_choro, ax = plt.subplots(figsize=(9, 7))
    gdf.plot(column='price', cmap='RdYlGn', legend=True, ax=ax, markersize=60)
    ax.set_title('House Prices — Spatial Distribution')
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    plt.tight_layout()
    fig_choro.savefig('../outputs/charts/png/03_spatial_choropleth.png',
                      dpi=150, bbox_inches='tight')
    print('Choropleth saved: ../outputs/charts/png/03_spatial_choropleth.png')
    plt.show()
except ImportError:
    print('geopandas not installed — choropleth map skipped.')
    print('Install with: pip install geopandas')
    print('Continuing with the rest of the notebook.')

## Part 4: Model Comparison Visualizations

When selecting among competing specifications, visual comparison is more intuitive
than comparing tables of numbers.

PanelBox provides:
- **Coefficient comparison** — side-by-side coefficients from multiple models
- **Forest plot** — confidence intervals in meta-analysis style
- **Fit comparison** — R², adjusted R² as bar charts
- **IC comparison** — AIC, BIC, HQIC side by side

In [ ]:
from panelbox.models.static import PooledOLS, FixedEffects, RandomEffects

# Generate a clean balanced panel (1 covariate for easy comparison)
df_comp       = generate_panel_data(n_individuals=40, n_periods=15, n_covariates=1, seed=42)
df_comp_reset = df_comp.reset_index()

# Pooled OLS
results_ols = PooledOLS(
    formula='y ~ x1',
    data=df_comp_reset,
    entity_col='entity',
    time_col='time',
).fit()

# Fixed Effects (within estimator)
results_fe = FixedEffects(
    formula='y ~ x1',
    data=df_comp_reset,
    entity_col='entity',
    time_col='time',
    entity_effects=True,
).fit()

# Random Effects (GLS with Swamy-Arora variance components)
results_re = RandomEffects(
    formula='y ~ x1',
    data=df_comp_reset,
    entity_col='entity',
    time_col='time',
).fit()

print('All 3 models estimated.')
print(f'OLS  β_x1 = {results_ols.params["x1"]:.4f}')
print(f'FE   β_x1 = {results_fe.params["x1"]:.4f}')
print(f'RE   β_x1 = {results_re.params["x1"]:.4f}')

### 4.1 Coefficient Comparison and Forest Plot

In [ ]:
from panelbox.visualization.api import create_comparison_charts

# Note: chart type strings are 'coefficients', 'forest_plot', 'fit_comparison', 'ic_comparison'
# forest_plot requires a single model — we generate it separately for each model
comparison = create_comparison_charts(
    results_list=[results_ols, results_fe, results_re],
    names=['Pooled OLS', 'Fixed Effects', 'Random Effects'],
    theme='academic',
    charts=['coefficients', 'fit_comparison', 'ic_comparison'],
)

# Forest plot requires single model — generate for FE model
forest = create_comparison_charts(
    results_list=[results_fe],
    names=['Fixed Effects'],
    theme='academic',
    charts=['forest_plot'],
)
if 'forest_plot' in forest:
    comparison['forest_plot'] = forest['forest_plot']

print(f'Charts generated: {list(comparison.keys())}')

In [ ]:
# Coefficient comparison — grouped bars, one group per variable
# Chart objects have a .figure attribute (Plotly Figure) and .to_png(), .save_image() methods
key = 'coefficients'
if key in comparison:
    comparison[key].figure.show()
    try:
        comparison[key].figure.write_image('../outputs/charts/png/03_comparison_coef.png',
                                           width=900, height=500)
        print('Coefficient comparison saved.')
    except Exception as e:
        print(f'Save failed (install kaleido: pip install kaleido): {e}')
else:
    print(f'Key "{key}" not found. Available: {list(comparison.keys())}')

In [ ]:
# Forest plot — meta-analysis style with confidence intervals per model
key = 'forest_plot'
if key in comparison:
    comparison[key].figure.show()
    try:
        comparison[key].figure.write_image('../outputs/charts/png/03_comparison_forest.png',
                                           width=750, height=400)
        print('Forest plot saved.')
    except Exception as e:
        print(f'Save failed: {e}')
else:
    print(f'Key "{key}" not found. Available: {list(comparison.keys())}')

In [ ]:
# Model fit comparison — R² and adjusted R² per model
key = 'fit_comparison'
if key in comparison:
    comparison[key].figure.show()
else:
    print(f'Key "{key}" not found. Available: {list(comparison.keys())}')

In [ ]:
# Information criteria comparison — AIC, BIC, HQIC per model
key = 'ic_comparison'
if key in comparison:
    comparison[key].figure.show()
else:
    print(f'Key "{key}" not found. Available: {list(comparison.keys())}')

### 4.2 Interactive Model Comparison — Dropdown

In [ ]:
import plotly.graph_objects as go

model_names   = ['Pooled OLS', 'Fixed Effects', 'Random Effects']
model_results = [results_ols, results_fe, results_re]

fig_drop = go.Figure()

for i, (name, res) in enumerate(zip(model_names, model_results)):
    params = res.params
    ci     = res.conf_int(alpha=0.05)
    x_vars = list(params.index)
    y_vals = list(params.values)
    err_p  = [ci.loc[p, 'upper'] - params[p] for p in x_vars]
    err_m  = [params[p] - ci.loc[p, 'lower'] for p in x_vars]

    fig_drop.add_trace(go.Bar(
        x=x_vars, y=y_vals,
        name=name,
        visible=(i == 1),     # show FE by default
        error_y=dict(type='data', array=err_p, arrayminus=err_m, visible=True),
        marker_color=['steelblue', 'coral', 'seagreen'][i],
    ))

# Dropdown menu
buttons = []
for i, name in enumerate(model_names):
    buttons.append(dict(
        label=name,
        method='update',
        args=[{'visible': [j == i for j in range(len(model_names))]}],
    ))

fig_drop.update_layout(
    title='Coefficients with 95% CI — use dropdown to switch model',
    xaxis_title='Variable',
    yaxis_title='Coefficient',
    template='plotly_white',
    font=dict(family='Arial', size=13),
    updatemenus=[dict(
        buttons=buttons, showactive=True,
        x=0.05, y=1.15, direction='down',
    )],
)
fig_drop.show()

fig_drop.write_html(
    '../outputs/reports/html/03_interactive_comparison.html',
    include_plotlyjs=True,
)
print('Interactive dashboard saved: ../outputs/reports/html/03_interactive_comparison.html')

In [ ]:
# --- Exercise 3 ---
# Extend the comparison to include a two-way FE model (entity_effects=True, time_effects=True).
# Add it to the dropdown plot above.
# Does the coefficient on x1 change significantly?

# YOUR CODE HERE


## Summary

| Part | Visualization | Key function / class |
|---|---|---|
| VAR | Impulse Response Functions | `plot_irf(irf_result, ...)` |
| VAR | FEVD (Stacked area) | `plot_fevd(fevd_result, ...)` |
| VAR | Stability (Unit circle) | `var_results.plot_stability()` |
| VAR | Granger Causality Network | `var_results.plot_causality_network()` |
| Quantile | Coefficient Path β(τ) | `QuantileVisualizer.coefficient_path()` |
| Quantile | Fan Chart | `QuantileVisualizer.fan_chart()` |
| Quantile | Interactive Dashboard | `InteractivePlotter.coefficient_dashboard()` |
| Spatial | Moran Scatterplot | `create_moran_scatterplot(values, W)` |
| Spatial | Weights Heatmap | `plot_spatial_weights_structure(W)` |
| Spatial | Weights Network | Plotly `Scatter` edge traces |
| Comparison | Coefficient Plot | `create_comparison_charts(..., charts=['coefficients'])` |
| Comparison | Forest Plot | `create_comparison_charts(..., charts=['forest_plot'])` |
| Comparison | IC Comparison | `create_comparison_charts(..., charts=['ic_comparison'])` |

**Next:** Notebook 04 — Automated Report Generation